### Study of Daily Volatility/Returns of Nifty 50

- Read the CSV data, convert it to a dataframe and calculate daily returns
- Calculate the mean of daily returns
- Plot a histogram of daily returns - to visualize a normal distribution
- Calculate measures like skewness and kurtosis

In [ ]:
## Imports: Necessary libraries
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.stats import kurtosis, skew

In [ ]:
## Function to calculate the daily returns
def get_daily_retruns(spot_df):
    ## OHLC aggregator function
    ohlc_aggregator = {
        "Open": "first",
        "High": "max",
        "Low": "min",
        "Close": "last",
    }
    ## Resampling of dataframe based on daily basis
    spot_df = spot_df.resample("D", on="Date").agg(ohlc_aggregator)
    spot_df.reset_index(inplace=True)
    spot_df.dropna(inplace=True)
    ## Calculate daily returns
    spot_df["Returns"] = round((spot_df["Close"].pct_change() * 100), 3)
    spot_df = spot_df[1:]
    return spot_df

In [ ]:
## Input file pahts
BASE_PATH = "F:\\MyProjects\\StockStrategies\\SectorIndex_1min_Data"
nifty_df = pd.read_csv(f"{BASE_PATH}\\NIFTY 50_minute.csv", index_col=0)
## Filtering out the duration
start_date = "2018-04-01"
end_date = "2023-03-31"
nifty_df = nifty_df[(nifty_df["Date"]>=start_date) & (nifty_df["Date"]<=end_date)]
nifty_df["Date"] = pd.to_datetime(nifty_df["Date"])
## Dropping of non-required columns
nifty_df.drop(columns="Volume", inplace=True)

In [ ]:
## Call the main function
nifty_daily_df = get_daily_retruns(spot_df=nifty_df)

In [ ]:
## Result DataFrame
nifty_daily_df.head()

In [ ]:
## Filter the extreme high volatilities dates
high_volatility = nifty_daily_df[(nifty_daily_df["Returns"]>5) | (nifty_daily_df["Returns"]<-5)]
print(f"Length of High Volatility Dataframe:", len(high_volatility))
high_volatility

In [ ]:
## In general days - After removing extreme points
nifty_daily_df = nifty_daily_df[(nifty_daily_df["Returns"]<=5) & (nifty_daily_df["Returns"]>=-5)]

In [ ]:
## Mean and Std. Deviation of Daily Returns
mean_df = round(nifty_daily_df["Returns"].mean(), 2)
std_df = round(nifty_daily_df["Returns"].std(), 2)
print("Mean of Daily Returns:", mean_df)
print("Std. Deviation of Daily Returns:", std_df)

#### Visualization of Graphs

In [ ]:
# Plot the histogram of Daily Returns
plt.figure(figsize=(10,5))
plt.hist(nifty_daily_df['Returns'], bins=75, color='blue', alpha=0.9)
plt.xlabel('Daily Returns (%)')
plt.ylabel('Frequency')
plt.title('Histogram of Nifty 50 Daily Returns')
plt.grid(True)
plt.show()

In [ ]:
# Calculate kurtosis and skewness
kurt = round(kurtosis(nifty_daily_df['Returns']), 2)
skewness = round(skew(nifty_daily_df['Returns']), 2)
print("Skewness of distribution:", skewness)
print("Kurtosis of distribution:", kurt)

#### Proof of Normal Distribution

In [ ]:
## 1st Standard Deviation
lower_st = (mean_df - (1 * std_df))
upper_st = (mean_df + (1 * std_df))
std_1_df = nifty_daily_df[(nifty_daily_df["Returns"]>=lower_st) & (nifty_daily_df["Returns"]<=upper_st)]
pct = round((len(std_1_df)/len(nifty_daily_df))*100, 2)
print(f"Daily Returns are in b/w the 1st standard deviation: {pct}")

In [ ]:
## 2nd Standard Deviation
lower_st = (mean_df - (2 * std_df))
upper_st = (mean_df + (2 * std_df))
std_2_df = nifty_daily_df[(nifty_daily_df["Returns"]>=lower_st) & (nifty_daily_df["Returns"]<=upper_st)]
pct = round((len(std_2_df)/len(nifty_daily_df))*100, 2)
print(f"Daily Returns are in b/w the 2nd standard deviation: {pct}")